# CMPE-257 Machine Learning - **RAG using Weaviate**




**Team Name:** Code Wizards <br/>
**Team Members:** <br/>
  Bhavya Hegde <br/>
  Blessy Dickson Daniel Moses <br/>
  Darshini Venkatesha Murthy Nag <br/>
  Sirisha Polisetty<br/><br/>


**Poet Chosen** <br/>
William Shakespeare



# Poems

In [ ]:
# Attempting to install various Python packages including NLP and data handling libraries, as well as a PDF manipulation tool.
!pip install -q transformers peft accelerate bitsandbytes safetensors sentencepiece streamlit weaviate-client langchain sentence-transformers tiktoken
!pip install PyPDF2 langchain
!pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 26.0 MB/s eta

In [ ]:
# Mount Google Drive in Google Colab to access files stored in it directly from the notebook.
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# fixing unicode error in google colab
import locale

locale.getpreferredencoding = lambda: "UTF-8"

# import dependencies
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)
from langchain.text_splitter import TokenTextSplitter
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.vectorstores import Weaviate
import weaviate

## Weaviate Cloud Services

In [ ]:
# Initialize a Weaviate client with an endpoint URL and API key for authentication to interact with the Weaviate database.
WEAVIATE_URL = "https://sjsusandbox-nc96n09s.weaviate.network"
WEAVIATE_API_KEY = "WTSt26QlTczkJ6OkEgW4ce6aDl1i8MilOgPw"

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

We will begin by defining the embedding model, which can be easily retrieved from HuggingFace using the following code:

In [ ]:
# specify embedding model (using huggingface sentence transformer)
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name, model_kwargs=model_kwargs
)

(…)851d5dd1af673670cdb299753/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)1af673670cdb299753/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)6e48e851d5dd1af673670cdb299753/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)48e851d5dd1af673670cdb299753/config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

(…)299753/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)851d5dd1af673670cdb299753/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)1af673670cdb299753/tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

(…)51d5dd1af673670cdb299753/train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

(…)6e48e851d5dd1af673670cdb299753/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)8e851d5dd1af673670cdb299753/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

## Ingest Scraped Poems  into Weaviate

In [ ]:
# Load poems from a CSV, split the text into chunks, and create a vector database with Weaviate for further processing.
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="/content/drive/MyDrive/LLM-Poem/William_Shakespeare.csv", source_column="Poem")
data = loader.load()
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=0)
split_docs = text_splitter.split_documents(data)
vector_db = Weaviate.from_documents(
    split_docs, embeddings, client=client, by_text=False
)

## Test the Vector Retriever

In [ ]:
# Conduct a similarity search for Shakespeare's love poems in the vector database, returning the top 3 results.
vector_db.similarity_search(
    "Which are the love poems by shakespeare", k=3)

[Document(page_content=' quenched in cool well by which from love fire took heat perpetual growing bath and healthful remedy for men diseased but my mistress thrall came there for cure and this by that prove love fire heats water water cools not love note the last group of poems all involve particular woman referred to as the dark lady because of discriptions of her contained within the poems save the last two which have to do with cupid but are generally regarded as being in the same vein as the poems that come before it the tone of the sonnets ranges from beatification to admonishment and describes multi faceted relationship that the speaker at different points both', metadata={'row': 7, 'source': 'cxxviiin the old age black was not counted fair or if it were it bore not beauty name but now is black beauty successive heir and beauty slandered with bastard shame for since each hand hath put on nature power fairing the foul with art false borrowed face sweet beauty hath no name no holy

## Setting up our Local LLM

In [ ]:
# specify model huggingface mode name
model_name = "darshini/MistralAI7BV1"

# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    :param model_name: Name or path of the model to be loaded.
    :return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

# function for initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    Initialize the tokenizer with the specified model_name.

    :param model_name: Name or path of the model for tokenizer initialization.
    :return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

# initialize tokenizer
tokenizer = initialize_tokenizer(model_name)
# load model
model = load_quantized_model(model_name)
# specify stop token ids
stop_token_ids = [0]

# build huggingface pipeline for using zephyr-7b-alpha
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2048,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

# specify the llm
llm = HuggingFacePipeline(pipeline=pipeline)

(…)I7BV1/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

(…)istralAI7BV1/resolve/main/tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

(…)ralAI7BV1/resolve/main/added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)BV1/resolve/main/special_tokens_map.json:   0%|          | 0.00/549 [00:00<?, ?B/s]

...

pytorch_model_00010-of-00010.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

pytorch_model_00011-of-00010.bin:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

(…)rded/resolve/main/generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/340M [00:00<?, ?B/s]

## Building a Conversation Chain
Now that we have our vector retrieval and th LLM ready, we can implement a retrieval-augmented chatbot in only a couple lines of code.

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vector_db.as_retriever()
)

Let’s now test how well it works:

In [ ]:
response = qa_chain.run(
    "can you list sad poems by William Shakespeare ")
print(response)

 A Lover's Complaint from off hill


Let’s try another one:

In [ ]:
response = qa_chain.run("can you list  3-4 Sonnets by William Shakespeare?")
print(response)

 1-2,3 The procreation sonnets 1-3 The first set of poems in the fair youth series in which the speaker encourages the young man to marry and father children in order to preserve beauty by passing it on to future generations. The sonnets in this group are all numbered in the range 1-12 and 13-20 and are found in the 1609


#News Articles

## Scraping News Articles

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import csv
import os

# List of Wikipedia URLs
urls = [
    'https://en.wikipedia.org/wiki/Spanish_Armada',
    'https://en.wikipedia.org/wiki/Elizabethan_era',
    'https://en.wikipedia.org/wiki/Roanoke_Colony',
    'https://en.wikipedia.org/wiki/Gunpowder_Plot',
    'https://en.wikipedia.org/wiki/Jamestown,_Virginia',
    'https://en.wikipedia.org/wiki/King_James_Version',
    'https://en.wikipedia.org/wiki/Thirty_Years%27_War',
    'https://en.wikipedia.org/wiki/Astronomia_nova',
    'https://en.wikipedia.org/wiki/Edo_period',
    'https://en.wikipedia.org/wiki/Globe_Theatre',
    'https://en.wikipedia.org/wiki/Sidereus_Nuncius',
    'https://en.wikipedia.org/wiki/Caribbean',
    'https://en.wikipedia.org/wiki/Don_Quixote',
    'https://en.wikipedia.org/wiki/Quebec_City',
    'https://en.wikipedia.org/wiki/Tokugawa_shogunate',
    'https://en.wikipedia.org/wiki/Chesapeake_Bay',
    'https://en.wikipedia.org/wiki/First_Folio',
    'https://en.wikipedia.org/wiki/Qing_dynasty',
    'https://en.wikipedia.org/wiki/European_exploration_of_Australia',
    'https://en.wikipedia.org/wiki/European_colonization_of_the_Americas',
    'https://en.wikipedia.org/wiki/Edict_of_Nantes',
    'https://en.wikipedia.org/wiki/Dutch_East_India_Company',
    'https://en.wikipedia.org/wiki/Battle_of_Lepanto',
    'https://en.wikipedia.org/wiki/History_of_the_Philippines_(1521%E2%80%931898)',
    'https://en.wikipedia.org/wiki/Essays_(Montaigne)',
    'https://en.wikipedia.org/wiki/Scientific_method',
    'https://en.wikipedia.org/wiki/Walter_Raleigh',
    'https://en.wikipedia.org/wiki/Treaty_of_Tordesillas',
    'https://en.wikipedia.org/wiki/Bank_of_Amsterdam',
    'https://en.wikipedia.org/wiki/Battle_of_Sekigahara'

]

# Path for the 'NewsArticles' folder in your mounted drive
folder_path = '/content/drive/MyDrive/NewsArticles'

# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# CSV file path for newsarticles.csv
csv_file_path = os.path.join(folder_path, 'NewsArticles.csv')

def process_wikipedia_article(url):
    # Fetch and parse the webpage
    source = urlopen(url).read()
    soup = BeautifulSoup(source, 'lxml')

    # Extract the page title
    title = soup.find('h1', {'id': 'firstHeading'}).get_text()

    # Extract text from paragraphs and headers
    paras = [paragraph.get_text() for paragraph in soup.find_all('p')]
    heads = [head.get_text() for head in soup.find_all('span', {'mw-headline'})]
    text = [val for pair in zip(paras, heads) for val in pair]
    article_text = ' '.join(text)
    article_text = re.sub(r"\[.*?\]+", '', article_text)
    article_text = article_text.replace('\n', '')

    return title, article_text

# Initialize CSV file with headers
with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Event', 'Article'])  # Write headers

# Append each article's data to the CSV file
for url in urls:
    title, article_text = process_wikipedia_article(url)

    # Prepare data for CSV
    data = [title, article_text]

    # Append data to CSV file
    with open(csv_file_path, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(data)

print('Data Scraped and is written into CSV file successfully')


Data Scraped and is written into CSV file successfully


## Ingest Scraped News Articles into Weaviate

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path="/content/drive/MyDrive/NewsArticles/NewsArticles.csv", source_column="Article",encoding="utf-8")
data = loader.load()
text_splitter = TokenTextSplitter(chunk_size=128, chunk_overlap=0)
split_docs = text_splitter.split_documents(data)
vector_db = Weaviate.from_documents(
    split_docs, embeddings, client=client, by_text=False
)

## Test the Vector Retriever

In [ ]:
import textwrap

In [ ]:
vector_db.similarity_search(
    "can youlist some major events", k=3)

[Document(page_content=' provision of 75 tons of silver bullion being sent from Spanish America on the Manila galleons. Financial constraints meant the 200-year-old fortifications in Manila did not see significant change after being first built by the early Spanish colonizers. External links', metadata={'row': 23, 'source': ' Spanish colonization Events/Artifacts Background (north to south) Conquest under Philip II Events/Artifacts Dutch attacks Artifacts British occupation of Manila The history of the Philippines from 1565 to 1898 is known as Spanish colonial period, during which the Philippine Islands were ruled as the Captaincy General of the Philippines within the Spanish East Indies, initially under the Viceroyalty of New Spain, based in Mexico City, until the independence of the Mexican Empire from Spain in 1821. This resulted in direct Spanish control during a period of governmental instability there. The Philippines was under direct royal governance  from 1821 to 1898. The open

## Building a Conversation Chain
Now that we have our vector retrieval and the LLM ready, we can implement a retrieval-augmented chatbot in only a couple lines of code.

In [ ]:
import textwrap

### Questions without RAG

In [ ]:
from transformers import pipeline,logging
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

template = """Question: {question}
Answer: """
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "can you list some major events happend during William Shakespeares lifetime?"


wrapped = textwrap.fill(llm_chain.run(question), width=80)
print(wrapped)

1574: William Shakespeare was born in Stratford-upon-Avon, England. 1592:
Shakespeare became a shareholder and actor in the Lord Chamberlain's Men, later
known as the King's Men. 1594: The first known performance of one of
Shakespeare's plays, "The Taming of the Shrew," took place. 1603: Shakespeare
became the first writer to be granted a patent by the Lord Chamberlain to
perform his plays at the Globe Theatre. 1611: Shakespeare was appointed the
first poet laureate of England by King James I. 1613: The Globe Theatre burned
down during a performance of Shakespeare's play "The Tempest." 1616: Shakespeare
retired from his career and went to Stratford-upon-Avon, where he died in 1616.


In [ ]:
question = "William Shakespeare  wrote a poem called Carpe Diem, how does this poem relate to the events that occurred between  1564 and 1616?"
wrapped = textwrap.fill(llm_chain.run(question), width=80)
print(wrapped)

 Carpe Diem is a Latin phrase that translates to "seize the day." This poem by
William Shakespeare is believed to have been written around 1599 and is included
in his play, The Rape of Lucrece. The poem is a message to live in the present
and not waste the current moment, but instead to make the most of it.   The
events that occurred between 1564 and 1616 saw significant changes and
developments in the world. The Renaissance began in the 14th century and was a
major cultural, artistic, and intellectual movement that had a profound impact
on European society. Shakespeare was born during the Renaissance and his work is
a reflection of the cultural changes occurring during this time. The Industrial
Revolution also began in the late 17th century, and Shakespeare's work was a
precursor to the modern novel, which became a popular form of literature during
this time.  Overall, the events that occurred between 1564 and 1616 were marked
by significant cultural and intellectual changes, and the 

In [ ]:
question = "William shakespear poem called sonnet 73, does this relate to any major events?"
wrapped = textwrap.fill(llm_chain.run(question), width=80)
print(wrapped)

  Sonnet 73 by William Shakespeare is not directly related to any major events.
It is a love poem that expresses the idea that beauty is only skin deep, while
true love is found in the heart and soul of the person. The poem is often used
at weddings and other romantic occasions as a way to express deep love and
devotion.


### Questions with RAG

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vector_db.as_retriever()
)

In [ ]:
response = qa_chain.run(
    "can you list some major events happend during William Shakespeares lifetime ")
wrapped = textwrap.fill(response, width=80)
print(wrapped)

 William Shakespeare lived during the Elizabethan era. Some major events that
happend during his lifetime include the Spanish Armada in 1588, when the Spanish
invaded England but were repelled, and the death of Queen Elizabeth I in 1603,
who was succeeded by King James I. Shakespeare was also a member of the Lord
Chamberlain's Men (later the King's Men), a theatre company that was active
during his lifetime.


In [ ]:
response = qa_chain.run("William shakespear poem called sonnet 73, does this relate to any major events?")
wrapped = textwrap.fill(response, width=80)
print(wrapped)

 No. The poem Sonnet 73 is not directly related to any major events, it's more
of a personal reflection of the poet on beauty in general, and the beauty of his
beloved in particular.


In [ ]:
response = qa_chain.run(
    "William Shakespeare  wrote a poem called Carpe Diem, how does this poem relate to the events that occurred between  1564 and 1616? ")
wrapped = textwrap.fill(response, width=80)
print(wrapped)

 Carpe Diem, also known as Seize the Day, is a poem written by William
Shakespeare in the form of a sonnet. The poem is a plea for living life to its
fullest and seizing opportunities while they last, since life is fleeting and
not to be wasted. The poem does not have any direct relation to the events that
occurred between 1564 and 1616, but it is a timeless message that is relevant to
all of human experience.


# Conclusion



1. We have successfully integrated a collection of news articles and poems into Weaviate, a vector database, for efficient information retrieval.

2. The chatbot's performance improved noticeably when using RAG, especially in handling queries related to the news articles.

3. A comparison between the chatbot's responses with and without RAG showed that RAG significantly enhanced the relevance and accuracy of the responses.

4. RAG enabled the chatbot to better understand and respond to queries by leveraging the context provided by the stored news articles.


